In [1]:
# Import library
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [0]:
import requests
import lxml
from bs4 import BeautifulSoup

In [3]:
# Read data
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"}
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url, headers=headers)
# Check status code
source.status_code

200

In [0]:
# Change to soup
soup = BeautifulSoup(source.content, "lxml")

In [5]:
# Find table
table = soup.find("table", class_="wikitable")

# Convert to dataframe
df = pd.read_html(str(table))
df = pd.DataFrame(df[0])
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [6]:
# drop nan
df.dropna(subset=["Neighborhood"], axis=0, inplace=True)
df.shape

(103, 3)

In [7]:
# Import coordinate file
coordinate = pd.read_csv("/content/drive/My Drive/Colab Notebooks/CAPSTONE/Geospatial_Coordinates.csv", header=0)
coordinate.head()
# Concate two dataframe
df = df.join(coordinate.set_index(["Postal code"]), on=["Postal code"]).reset_index(drop=True)
# Drop Postal code column
df.drop(["Postal code"], axis=1, inplace=True)
df.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


In [8]:
print("The data frame got {} boroughs and {} neighborhoods".format(len(df["Borough"].unique()), df.shape[0]))

The data frame got 10 boroughs and 103 neighborhoods


**Use Geopy library to to get latitude and longitude values of Toronto**

In [9]:
address = "Toronto City"
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geograpical coordinate of Toronto City are {}, {}".format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347


**Create a map of Toronto with neighborhoods superimposed on top**

In [10]:
# Create map of Toronto with latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# Add markers to map
for lat, lng, borough, neighborhood in zip(df["Latitude"], df["Longitude"], df["Borough"], df["Neighborhood"]):
  label = '{}, {}'.format(neighborhood, borough)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker(
      [lat, lng],
      radius=5,
      popup=label,
      color="blue",
      fill=True,
      fill_color='#3186cc',
      fill_opacity=0.7,
      parse_html=False).add_to(map_toronto)

map_toronto

**I decide to work with boroughs that contain Toronto word**

In [11]:
df["Borough"].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [12]:
# Built new dataframe have Borough contain Toronto
df = df.loc[df["Borough"].map(lambda x:"Toronto" in x),:]
df.head()

,Borough,Neighborhood,Latitude,Longitude
2,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
4,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
9,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,Downtown Toronto,St. James Town,43.651494,-79.375418
19,East Toronto,The Beaches,43.676357,-79.293031


**Define Foursquare Credential and Version**

In [13]:
CLIENT_ID = "EKN22PKJXDECHH3XZ5CEPCLOUJQU41LTJ4GKT2OCA42SOJOV"
CLIENT_SECRET = "MXVBAERR52VFH5J4BDOXM3LU4YVUS342NOK5YRXZDPNCDCYE"
VERSION = "20200409"
print("Your Credential:")
print("CLIENT_ID:{}".format(CLIENT_ID))
print("CLIENT_SECRET:{}".format(CLIENT_SECRET))

Your Credential:
CLIENT_ID:EKN22PKJXDECHH3XZ5CEPCLOUJQU41LTJ4GKT2OCA42SOJOV
CLIENT_SECRET:MXVBAERR52VFH5J4BDOXM3LU4YVUS342NOK5YRXZDPNCDCYE


**Let explore the first Neighborhood**

In [14]:
neigh_name = df.iloc[0,1]
neigh_lat = df.iloc[0,2]
neigh_long = df.iloc[0,3]
print("The first neighborhood is:{} - Latitude:{} - Longitude:{}".format(neigh_name, neigh_lat, neigh_long))

The first neighborhood is:Regent Park / Harbourfront - Latitude:43.6542599 - Longitude:-79.3606359


**Let get the top 100 veneus that are in Regent Park / Harbourfront with radius 500**

In [15]:
LIMIT = 100
RADIUS = 500
url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(CLIENT_ID, CLIENT_SECRET, VERSION, neigh_lat, neigh_long, RADIUS, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=EKN22PKJXDECHH3XZ5CEPCLOUJQU41LTJ4GKT2OCA42SOJOV&client_secret=MXVBAERR52VFH5J4BDOXM3LU4YVUS342NOK5YRXZDPNCDCYE&v=20200409&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [16]:
# Send the GET request and examine result
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e90214471c428001b03e5e3'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-54ea41ad498e9a11e9e13308-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16a941735',
         'name': 'Bakery',
         'pluralName': 'Bakeries',
         'primary': True,
         'shortName': 'Bakery'}],
       'id': '54ea41ad498e9a11e9e13308',
       'location': {'address': '362 King St E',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Trinity St',
        'distance': 143,
        'formattedAddress': ['362 King St E (Trinity St)',
         'Toronto ON M5A 1K9',
         'Canada'],
        'labeledLatLngs': [{'label': 'display',
 

In [0]:
# Define function to extract category of the venue
def get_category_type(row):
  try:
    categories_list = row["categories"]
  except:
    categories_list = row["venue.categories"]

  if len(categories_list) == 0:
    return None
  else:
    return categories_list[0]["name"]

In [18]:
# Ready to clean json and structure it into dataframe
venues = results["response"]["groups"][0]["items"]
nearby_venues = json_normalize(venues)

# Filter columns
filtered_cols = ["venue.name", "venue.categories", "venue.location.lat", "venue.location.lng"]
nearby_venues = nearby_venues[filtered_cols]

# Extract category name
nearby_venues["venue.categories"] = nearby_venues.apply(get_category_type, axis=1)

# Clean column name
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


In [19]:
print("Total venues is {}".format(nearby_venues.shape[0]))

Total venues is 46


**2. EXPLORE NEIGHBORHOODS IN DOWNTOWN, WEST, EAST AND CENTRAL TORONTO**

In [0]:
# Define function to get venue from every neighborhood
def GetNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
  venues_list = []
  for name, lat, lng in zip(names, latitudes, longitudes):
    print(name)

    # Create API request
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, limit)

    # make a GET request
    try:
      results = requests.get(url).json()["response"]["groups"][0]["items"]
    except:
      results = None

    # return only relevant information
    venues_list.append([(name, lat, lng, v["venue"]["name"], v["venue"]["location"]["lat"], v["venue"]["location"]["lng"], v["venue"]["categories"][0]["name"]) for v in results])

  # Build dataframe
  nearby_venues = pd.DataFrame(item for venue_list in venues_list for item in venue_list)

  nearby_venues.columns = ["Neighborhood", "Neighborhood Latitude", "Neighborhood Longitude", "Venue", "Venue Latitude", "Venue Longitude", "Venue Category"]

  return(nearby_venues)

In [23]:
# Get information 100 venues of neighborhood in Toronto
toronto_venues = GetNearbyVenues(df["Neighborhood"], df["Latitude"], df["Longitude"])

Regent Park / Harbourfront
Queen's Park / Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
The Danforth West / Riverdale
Toronto Dominion Centre / Design Exchange
Brockton / Parkdale Village / Exhibition Place
India Bazaar / The Beaches West
Commerce Court / Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park / The Junction South
North Toronto West
The Annex / North Midtown / Yorkville
Parkdale / Roncesvalles
Davisville
University of Toronto / Harbord
Runnymede / Swansea
Moore Park / Summerhill East
Kensington Market / Chinatown / Grange Park
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst  Quay / South Niagara / Island airport
Rosed

In [24]:
toronto_venues.shape

(1658, 7)

In [25]:
toronto_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park / Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent Park / Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent Park / Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Regent Park / Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Regent Park / Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
5,Regent Park / Harbourfront,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
6,Regent Park / Harbourfront,43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park
7,Regent Park / Harbourfront,43.65426,-79.360636,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
8,Regent Park / Harbourfront,43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site
9,Regent Park / Harbourfront,43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub


In [26]:
print("Number venues by neighborhood:")
toronto_venues.groupby(["Neighborhood"]).count()["Venue"]

Number venues by neighborhood:


Neighborhood
Berczy Park                                                                                                           56
Brockton / Parkdale Village / Exhibition Place                                                                        22
Business reply mail Processing CentrE                                                                                 17
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst  Quay / South Niagara / Island airport     17
Central Bay Street                                                                                                    72
Christie                                                                                                              18
Church and Wellesley                                                                                                  78
Commerce Court / Victoria Hotel                                                                                      100
Davisville         

In [27]:
print("There are {} unique categories".format(len(toronto_venues["Venue Category"].unique())))

There are 234 unique categories


**3. ANALYZE EACH NEIGHBOORHOOD**

In [28]:
# One hot encoding
toronto_onehot = pd.get_dummies(toronto_venues["Venue Category"], prefix="", prefix_sep="")

# Add neighborhood column to dataframe
toronto_onehot["Neighborhood"] = toronto_venues["Neighborhood"]

toronto_onehot.head()

,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [29]:
neigh_col_idx = toronto_onehot.columns.tolist().index("Neighborhood")
neigh_col_idx

162

In [30]:
# Move neighborhood column to first position
fixed_cols = [toronto_onehot.columns[neigh_col_idx]] + list(toronto_onehot.columns[:neigh_col_idx]) + list(toronto_onehot.columns[(neigh_col_idx+1):])
len(fixed_cols)

234

In [31]:
 toronto_onehot = toronto_onehot[fixed_cols]
 toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,

In [32]:
# Group by neighborhood and calculate mean of the frequency of occurence of each category
toronto_grouped = toronto_onehot.groupby("Neighborhood").mean().reset_index()
toronto_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.017857,0.0,0.0,0.0,0.000000,0.017857,0.0,0.017857,0.035714,0.0,0.000000,0.0,0.017857,0.017857,0.0,0.035714,0.0,0.0,0.017857,0.0,0.000000,0.0,0.000000,0.0,0.017857,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.017857,0.035714,0.0,0.0,0.0,0.0,0.035714,0.000000,0.0,0.0,0.000000,0.0,0.035714,0.053571,0.0,0.0,0.0,0.0,0.0,0.017857,0.000000,0.017857,0.000000,0.017857,0.0,0.0,0.017857,0.0,0.0,0.0,0.0,0.017857,0.000000,0.017857,0.000000,0.0,0.0,0.0,0.000000,0.0,0.017857,0.0,0.0,0.0,0.000000,0.035714,0.000000,0.0,0.0,0.017857,0.0,0.0,0.0,0.0,0.0,0.0,0.017857,

In [33]:
# Print each neighborhood along with top 5 most common
for i in range(toronto_grouped.shape[0]):
  print(toronto_grouped.iloc[i,0])
  s = toronto_grouped.iloc[i,:]
  s = s[1:].sort_values(ascending=False)[:5]
  print(s)
  print("\n")

Berczy Park
Coffee Shop    0.0535714
Bakery         0.0357143
Beer Bar       0.0357143
Café           0.0357143
Cheese Shop    0.0357143
Name: 0, dtype: object


Brockton / Parkdale Village / Exhibition Place
Café               0.136364
Coffee Shop       0.0909091
Breakfast Spot    0.0909091
Grocery Store     0.0454545
Stadium           0.0454545
Name: 1, dtype: object


Business reply mail Processing CentrE
Light Rail Station     0.117647
Auto Workshop         0.0588235
Comic Shop            0.0588235
Pizza Place           0.0588235
Recording Studio      0.0588235
Name: 2, dtype: object


CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst  Quay / South Niagara / Island airport
Airport Service      0.176471
Airport Lounge       0.117647
Airport Terminal     0.117647
Sculpture Garden    0.0588235
Boutique            0.0588235
Name: 3, dtype: object


Central Bay Street
Coffee Shop             0.166667
Italian Restaurant     0.0694444
Café                   0.0416

In [0]:
# Define function to get most common categories every neighborhood
def return_most_common_categories(row, num_top_venues):
  row_categories = row.iloc[1:] # exclude name of neighborhood is first column
  row_categories_sorted = row_categories.sort_values(ascending=False)
  return row_categories_sorted.index.values[0:num_top_venues]

In [0]:
indicator = ["st", "nd", "rd"]
num_top_venues = 10
cols = ["Neighborhood"]
# Create column according to num top venues
for i in np.arange(num_top_venues):
  try:
    cols.append("{}{} Most Common Category".format(i+1, indicator[i]))
  except:
    cols.append("{}th Most Common Category".format(i+1))

In [36]:
cols

['Neighborhood',
 '1st Most Common Category',
 '2nd Most Common Category',
 '3rd Most Common Category',
 '4th Most Common Category',
 '5th Most Common Category',
 '6th Most Common Category',
 '7th Most Common Category',
 '8th Most Common Category',
 '9th Most Common Category',
 '10th Most Common Category']

***Create DataFrane after sorted by Most Common Category***

In [37]:
neighborhoods_venues_sorted = pd.DataFrame(columns=cols)
neighborhoods_venues_sorted["Neighborhood"] = toronto_grouped["Neighborhood"]
for i in np.arange(toronto_grouped.shape[0]):
  neighborhoods_venues_sorted.iloc[i,1:] = return_most_common_categories(toronto_grouped.iloc[i,:], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category,8th Most Common Category,9th Most Common Category,10th Most Common Category
0,Berczy Park,Coffee Shop,Bakery,Beer Bar,Café,Cheese Shop,Italian Restaurant,Cocktail Bar,Seafood Restaurant,Farmers Market,Restaurant
1,Brockton / Parkdale Village / Exhibition Place,Café,Coffee Shop,Breakfast Spot,Grocery Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store,Bakery
2,Business reply mail Processing CentrE,Light Rail Station,Auto Workshop,Comic Shop,Pizza Place,Recording Studio,Restaurant,Burrito Place,Brewery,Skate Park,Smoke Shop
3,CN Tower / King and Spadina / Railway Lands / ...,Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Boutique,Bar,Rental Car Location,Coffee Shop,Boat or Ferry,Harbor / Marina
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Japanese Restaurant,Salad Place,Gym / Fitness Center,Ice Cream Shop,Spa,Thai Restaurant


**4. CLUSTER NEIGHBORHOOD**

In [38]:
# set number of cluster
kcluster = 5

toronto_grouped_clustering = toronto_grouped.drop("Neighborhood", axis=1)

# run k-mean clustering
kmeans = KMeans(n_clusters=kcluster, random_state=42).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [39]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, "Cluster Labels", kmeans.labels_)

toronto_merged = df

# toronto_merged join with labels column
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index("Neighborhood"), on="Neighborhood")

toronto_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category,8th Most Common Category,9th Most Common Category,10th Most Common Category
2,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Mexican Restaurant,Breakfast Spot,Café,Theater,Event Space,Chocolate Shop
4,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,0,Coffee Shop,Diner,Yoga Studio,Park,Beer Bar,Boutique,Sandwich Place,Burger Joint,Burrito Place,Café
9,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Japanese Restaurant,Café,Cosmetics Shop,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Theater,Diner
15,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cocktail Bar,Italian Restaurant,Beer Bar,Restaurant,American Restaurant,Hotel,Bakery,Cosmetics Shop
19,East Toronto,The Beaches,43.676357,-79.293031,3,Trail,Pub,Health Food Store,Doner Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio


***Check whether neighborhood in df but not get information when make GET request***

In [41]:
# Find neighborhood not cluster 
na_values = toronto_merged.loc[toronto_merged["Cluster Labels"].isnull(),:]["Neighborhood"]
na_values

Series([], Name: Neighborhood, dtype: object)

In [42]:
# Move Cluster labels column to the last position
label_idx = toronto_merged.columns.tolist().index("Cluster Labels")
fixed_cols =  list(toronto_merged.columns[:label_idx]) + list(toronto_merged.columns[(label_idx+1):]) + [toronto_merged.columns[label_idx]]
toronto_merged = toronto_merged[fixed_cols]
toronto_merged.head()

,Borough,Neighborhood,Latitude,Longitude,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category,8th Most Common Category,9th Most Common Category,10th Most Common Category,Cluster Labels
2,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,Coffee Shop,Park,Bakery,Pub,Mexican Restaurant,Breakfast Spot,Café,Theater,Event Space,Chocolate Shop,0
4,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,Coffee Shop,Diner,Yoga Studio,Park,Beer Bar,Boutique,Sandwich Place,Burger Joint,Burrito Place,Café,0
9,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,Clothing Store,Coffee Shop,Japanese Restaurant,Café,Cosmetics Shop,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Theater,Diner,0
15,Downtown Toronto,St. James Town,43.651494,-79.375418,Coffee Shop,Café,Cocktail Bar,Italian Restaurant,Beer Bar,Restaurant,American Restaurant,Hotel,Bakery,Cosmetics Shop,0
19,East Toronto,The Beaches,43.676357,-79.293031,Trail,Pub,Health Food Store,Doner Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio,3


***Visualize Result***

In [0]:
# Need convert label colimn to integer type
toronto_merged["Cluster Labels"] = toronto_merged["Cluster Labels"].astype("int")

In [44]:
# create map
map_cluster = folium.Map(location=[latitude, longitude])

# set color scheme for the clusters
x = np.arange(kcluster)
ys = [i + x + (i*x)**2 for i in range(kcluster)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lng, poi, cluster in zip(toronto_merged["Latitude"], toronto_merged["Longitude"], toronto_merged["Neighborhood"], toronto_merged["Cluster Labels"]):
  label = folium.Popup(str(poi) + "Cluster" + str(cluster), parse_html=True)
  folium.CircleMarker(
        location=[lat, lng],
        radius=5, 
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_cluster)

map_cluster

**5. EXAMINE CLUSTER**

*Cluster 1*

In [45]:
toronto_merged.loc[toronto_merged["Cluster Labels"]==0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category,8th Most Common Category,9th Most Common Category,10th Most Common Category,Cluster Labels
2,Regent Park / Harbourfront,Park,Bakery,Pub,Mexican Restaurant,Breakfast Spot,Café,Theater,Event Space,Chocolate Shop,0
4,Queen's Park / Ontario Provincial Government,Diner,Yoga Studio,Park,Beer Bar,Boutique,Sandwich Place,Burger Joint,Burrito Place,Café,0
9,"Garden District, Ryerson",Coffee Shop,Japanese Restaurant,Café,Cosmetics Shop,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Theater,Diner,0
15,St. James Town,Café,Cocktail Bar,Italian Restaurant,Beer Bar,Restaurant,American Restaurant,Hotel,Bakery,Cosmetics Shop,0
20,Berczy Park,Bakery,Beer Bar,Café,Cheese Shop,Italian Restaurant,Cocktail Bar,Seafood Restaurant,Farmers Market,Restaurant,0
24,Central Bay Street,Italian Restaurant,Café,Sandwich Place,Japanese Restaurant,Salad Place,Gym / Fitness Center,Ice Cream Shop,Spa,Thai Restaurant,0
25,Christie,Café,Park,Nightclub,Coffee Shop,Italian Restaurant,Baby Store,Athletics & Sports,Diner,Candy Store,0
30,Richmond / Adelaide / King,Café,Restaurant,Gym,Bakery,Hotel,Thai Restaurant,Bookstore,Steakhouse,Bar,0
31,Dufferin / Dovercourt Village,Pharmacy,Bank,Middle Eastern Restaurant,Bar,Supermarket,Café,Grocery Store,Gym / Fitness Center,Brewery,0
36,Harbourfront East / Union Station / Toronto Is...,Aquarium,Hotel,Restaurant,Italian Restaurant,Café,Fried Chicken Joint,Scenic Lookout,Brewery,Sporting Goods Shop,0


In [46]:
toronto_merged.loc[toronto_merged["Cluster Labels"]==1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category,8th Most Common Category,9th Most Common Category,10th Most Common Category,Cluster Labels
62,Roselawn,Garden,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,1


In [47]:
toronto_merged.loc[toronto_merged["Cluster Labels"]==2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category,8th Most Common Category,9th Most Common Category,10th Most Common Category,Cluster Labels
83,Moore Park / Summerhill East,Park,Tennis Court,Summer Camp,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center,Yoga Studio,2
91,Rosedale,Playground,Trail,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,2


In [48]:
toronto_merged.loc[toronto_merged["Cluster Labels"]==3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category,8th Most Common Category,9th Most Common Category,10th Most Common Category,Cluster Labels
19,The Beaches,Pub,Health Food Store,Doner Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio,3


In [49]:
toronto_merged.loc[toronto_merged["Cluster Labels"]==4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category,8th Most Common Category,9th Most Common Category,10th Most Common Category,Cluster Labels
61,Lawrence Park,Swim School,Bus Line,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,4
68,Forest Hill North & West,Jewelry Store,Trail,Sushi Restaurant,Bus Line,Yoga Studio,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,4
